<a href="https://colab.research.google.com/github/plaban1981/Hugging_Face_transformers_topics/blob/main/Medical_Domain_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install --quiet transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.3 MB/s eta 0:00:00


In [2]:
! pip install --quiet sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.9 MB/s eta 0:00:00


In [48]:
import torch 
from transformers import pipeline
device = "cuda" if torch.cuda.is_available() else "cpu"
summarizer = pipeline("summarization", model='facebook/bart-large-cnn',min_length=75,max_length=500)

In [35]:
text = """Telemedicine and health policy: A systematic review

Highlights
Frequently mentioned facilitators were access, convenience, population health, mobile technology care, and self-efficacy.
Frequently mentioned barriers were the increased cost to providers, patient privacy, technical literacy, state licensing, and data security.
Health policies can increase access, improve population health, and remove constraints to reach patients in developing countries.
Abstract
Background
Telemedicine diagnoses and treats patients remotely via telecommunications technology all over the world. Telemedicine becomes more prevalent as providers recognize the benefits, patients receive increased access and payers see the reduction in cost of care.

Objective
Telemedicine studies have shown success in limiting geographical constraints, time spent, and costs incurred by patients with positive health outcomes across medical specialties. The aim of this review is to evaluate the implications of telemedicine and health policies.

Methods
An assessment of the literature in four databases was made on content germane to health policy implications of telemedicine. From the results of the search, 48 publications were kept for analysis.

Results
The fifteen facilitators mentioned most often were increased access, increased convenience, improved population health, care enabled through mobile technology, self-efficacy, increased patient-to-provider communication, cost advantages, efficacy of modality, increased health outcomes, reaches developing countries, increased quality, a positive previous experience, and a secure means of care. The twelve barriers mentioned most often were the increased cost to providers, patient privacy, technical literacy, state licensing, data security, socioeconomics, limited reimbursements, issues of interoperability, patient safety, less personal means of care, misaligned incentives, and ethical concerns.

Conclusions
Telemedicine has the potential for growth and adoption, however, there are several implications and barriers of health policy surrounding telemedicine that make it difficult to adopt. Policies will likely encourage and incentivize its spread and use. Future research should focus on standardization of telemedicine and new policies and incentives that encourage its use.
"""

In [49]:
summarizer(text,truncation=True)

Your max_length is set to 500, but you input_length is only 437. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=218)


[{'summary_text': 'Health policies can increase access, improve population health, and remove constraints to reach patients in developing countries. The twelve barriers mentioned most often were the increased cost to providers, patient privacy, technical literacy, state licensing, data security, socioeconomics, limited reimbursements. Future research should focus on standardization of telemedicine and new policies and incentives that encourage its use.'}]

## Abstractive Text Summarization

In [4]:
import torch
import transformers

In [5]:
from transformers import T5Tokenizer,T5ForConditionalGeneration,T5Config
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("t5-base")

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [50]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
#
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

In [51]:
model = model.to(device)

In [36]:
preprocessed_text = text.strip().replace("\n","")

In [37]:
preprocessed_text

'Telemedicine and health policy: A systematic reviewHighlightsFrequently mentioned facilitators were access, convenience, population health, mobile technology care, and self-efficacy.Frequently mentioned barriers were the increased cost to providers, patient privacy, technical literacy, state licensing, and data security.Health policies can increase access, improve population health, and remove constraints to reach patients in developing countries.AbstractBackgroundTelemedicine diagnoses and treats patients remotely via telecommunications technology all over the world. Telemedicine becomes more prevalent as providers recognize the benefits, patients receive increased access and payers see the reduction in cost of care.ObjectiveTelemedicine studies have shown success in limiting geographical constraints, time spent, and costs incurred by patients with positive health outcomes across medical specialties. The aim of this review is to evaluate the implications of telemedicine and health po

In [38]:
input_text = "summarize:"+preprocessed_text

In [39]:
len(input_text.split())

288

In [40]:
torch.cuda.is_available()

True

In [41]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [52]:
tokenized_text = tokenizer.encode(input_text,return_tensors="pt",max_length=1024,truncation=True).to(device)

In [53]:
summary = model.generate(tokenized_text,min_length=30,max_length=500)
summary_text = tokenizer.decode(summary[0],skip_special_tokens=True)

In [54]:
".".join([text.strip().capitalize() for text in summary_text.split(".")])

'Health policies can increase access, improve population health, and remove constraints to reach patients in developing countries.Frequently mentioned barriers were the increased cost to providers, patient privacy, technical literacy, state licensing, and data security.'

In [55]:
!pip install -q bert-extractive-summarizer

In [68]:
import spacy
import textwrap
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest
punctuation += '\n' 
stopwords = list(STOP_WORDS)

reduction_rate = 0.1  #defines how small the output summary should be compared with the input
nlp_pl = spacy.load('en_core_web_sm')     #process original text according with the Spacy nlp pipeline for english
document = nlp_pl(preprocessed_text)                   #doc object

sentence_tokens = [sent for sent in document.sents]

In [69]:
len(sentence_tokens)

14

In [72]:
tokens = [token.text for token in document] #tokenized text

word_frequencies = {}
for word in document:
    if word.text.lower() not in stopwords:
        if word.text.lower() not in punctuation:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1

max_frequency = max(word_frequencies.values())
print(max_frequency)

for word in word_frequencies.keys():
    word_frequencies[word] = word_frequencies[word]/max_frequency

print(word_frequencies)

9
{'Telemedicine': 0.2222222222222222, 'health': 1.0, 'policy': 0.3333333333333333, 'systematic': 0.1111111111111111, 'reviewHighlightsFrequently': 0.1111111111111111, 'mentioned': 0.4444444444444444, 'facilitators': 0.2222222222222222, 'access': 0.4444444444444444, 'convenience': 0.2222222222222222, 'population': 0.3333333333333333, 'mobile': 0.2222222222222222, 'technology': 0.3333333333333333, 'care': 0.5555555555555556, 'self': 0.2222222222222222, 'efficacy': 0.3333333333333333, 'Frequently': 0.1111111111111111, 'barriers': 0.3333333333333333, 'increased': 0.8888888888888888, 'cost': 0.4444444444444444, 'providers': 0.3333333333333333, 'patient': 0.4444444444444444, 'privacy': 0.2222222222222222, 'technical': 0.2222222222222222, 'literacy': 0.2222222222222222, 'state': 0.2222222222222222, 'licensing': 0.2222222222222222, 'data': 0.2222222222222222, 'security': 0.2222222222222222, 'Health': 0.1111111111111111, 'policies': 0.3333333333333333, 'increase': 0.1111111111111111, 'improve'

In [73]:
def get_sentence_scores(sentence_tok, len_norm=True):
  sentence_scores = {}
  for sent in sentence_tok:
      word_count = 0
      for word in sent:
          if word.text.lower() in word_frequencies.keys():
              word_count += 1
              if sent not in sentence_scores.keys():
                  sentence_scores[sent] = word_frequencies[word.text.lower()]
              else:
                  sentence_scores[sent] += word_frequencies[word.text.lower()]
      if len_norm:
        sentence_scores[sent] = sentence_scores[sent]/word_count
  return sentence_scores
                
sentence_scores = get_sentence_scores(sentence_tokens,len_norm=False)        #sentence scoring without lenght normalization
sentence_scores_rel = get_sentence_scores(sentence_tokens,len_norm=True)

In [74]:
def get_summary(sentence_sc, rate):
  summary_length = int(len(sentence_sc)*rate)
  summary = nlargest(summary_length, sentence_sc, key = sentence_sc.get)
  final_summary = [word.text for word in summary]
  summary = ' '.join(final_summary)
  return summary
#the first output shows verbose and content-heavy sentences, while the second one is much more concise.
print("- NON_REL: "+ get_summary(sentence_scores, reduction_rate))
print("- REL: "+ get_summary(sentence_scores_rel, reduction_rate))

- NON_REL: ResultsThe fifteen facilitators mentioned most often were increased access, increased convenience, improved population health, care enabled through mobile technology, self-efficacy, increased patient-to-provider communication, cost advantages, efficacy of modality, increased health outcomes, reaches developing countries, increased quality, a positive previous experience, and a secure means of care.
- REL: Telemedicine and health policy: A systematic reviewHighlightsFrequently mentioned facilitators were access, convenience, population health, mobile technology care, and self-efficacy.


In [75]:
from summarizer import Summarizer
model = Summarizer()

summary_length = int(len(sentence_tokens)*reduction_rate)
print(summary_length)
result = model(preprocessed_text, num_sentences=summary_length, min_length=400)

print(''.join(result))

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


1
ResultsThe fifteen facilitators mentioned most often were increased access, increased convenience, improved population health, care enabled through mobile technology, self-efficacy, increased patient-to-provider communication, cost advantages, efficacy of modality, increased health outcomes, reaches developing countries, increased quality, a positive previous experience, and a secure means of care.


In [76]:
result

'ResultsThe fifteen facilitators mentioned most often were increased access, increased convenience, improved population health, care enabled through mobile technology, self-efficacy, increased patient-to-provider communication, cost advantages, efficacy of modality, increased health outcomes, reaches developing countries, increased quality, a positive previous experience, and a secure means of care.'

In [79]:
result = model(preprocessed_text,min_length=400)

print(''.join(result.replace('Results',"")))

The fifteen facilitators mentioned most often were increased access, increased convenience, improved population health, care enabled through mobile technology, self-efficacy, increased patient-to-provider communication, cost advantages, efficacy of modality, increased health outcomes, reaches developing countries, increased quality, a positive previous experience, and a secure means of care.


In [84]:
from summarizer import TransformerSummarizer
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
full = ''.join(GPT2_model(preprocessed_text, min_length=60))
print(full)

Telemedicine and health policy: A systematic reviewHighlightsFrequently mentioned facilitators were access, convenience, population health, mobile technology care, and self-efficacy. ObjectiveTelemedicine studies have shown success in limiting geographical constraints, time spent, and costs incurred by patients with positive health outcomes across medical specialties. Policies will likely encourage and incentivize its spread and use.


## Reading PDF

In [81]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [82]:
! pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 52.4 MB/s eta 0:00:00


In [ ]:
import fitz
from transformers import pipeline

# Load the text extraction pipeline
text_extractor = pipeline("text2text-generation", model="t5-small")

# Load the PDF document using PyMuPDF
doc = fitz.open("example.pdf")

# Extract text from each page of the PDF
text = ""
for page in doc:
    text += page.get_text()

# Use the text2text-generation pipeline to clean up the extracted text
clean_text = text_extractor(text, max_length=1024, do_sample=False)[0]['generated_text']

# Print the cleaned-up text
print(clean_text)
